In [5]:
import geopandas as gpd
import osmnx as ox
import networkx as nx
import folium
from shapely.geometry import Point

# Step 1: Simulate an ambulance's GPS location (incident site)
incident_location = Point(-80.8431, 35.2271)  # Charlotte, NC coordinates

# Step 2: Load hospital locations (sample data)
hospitals = gpd.GeoDataFrame(
    {'name': ['Hospital A', 'Hospital B'],
     'geometry': [Point(-80.8400, 35.2300), Point(-80.8500, 35.2200)]},
    crs='EPSG:4326'
)

# Step 3: Get the nearest hospital
hospitals['distance'] = hospitals.geometry.distance(incident_location)
nearest_hospital = hospitals.loc[hospitals['distance'].idxmin()]

# Step 4: Download road network from OpenStreetMap
G = ox.graph_from_point((35.2271, -80.8431), dist=2000, network_type='drive')

# Step 5: Find the nearest nodes in the network
orig_node = ox.distance.nearest_nodes(G, X=incident_location.x, Y=incident_location.y)
dest_node = ox.distance.nearest_nodes(G, X=nearest_hospital.geometry.x, Y=nearest_hospital.geometry.y)

# Step 6: Compute the shortest path
shortest_route = nx.shortest_path(G, orig_node, dest_node, weight='length')

# Step 7: Convert route to a list of lat/lon points
route_map = ox.plot_graph_route_folium(G, shortest_route, route_linewidth=6)

# Step 8: Add markers to the map
folium.Marker(
    location=[incident_location.y, incident_location.x],
    popup='Incident Location',
    icon=folium.Icon(color='red')
).add_to(route_map)

folium.Marker(
    location=[nearest_hospital.geometry.y, nearest_hospital.geometry.x],
    popup=nearest_hospital['name'],
    icon=folium.Icon(color='blue')
).add_to(route_map)

# Step 9: Save and display the map
route_map.save("ambulance_route.html")
print("Map saved as ambulance_route.html")


C:\Users\gangl\AppData\Local\Temp\ipykernel_34312\475362532.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hospitals['distance'] = hospitals.geometry.distance(incident_location)


AttributeError: module 'osmnx' has no attribute 'plot_graph_route_folium'